In [1]:
%cd C:/Users/Mathieu/Desktop/Projets/Benter

C:\Users\Mathieu\Desktop\Projets\Benter


In [2]:
%matplotlib inline
import collections
import functools

from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
import seaborn as sns
import pandas as pd
from tqdm import tqdm


from utils import import_data
from winning_horse_models.logistic_regression import LogisticRegressionModel
from constants import PMU_BETTINGS, SOURCE_PMU
from utils.wagering_stategy import compute_expected_return, compute_scenario, plot_scenario
import wagering_stategies
from wagering_stategies import factories as wagering_factories

from matplotlib import style
style.use('ggplot')

C:\Users\Mathieu\Anaconda3\envs\deeplearning\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Mathieu\Anaconda3\envs\deeplearning\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Mathieu\Anaconda3\envs\deeplearning\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Mathieu\Anaconda3\envs\deeplearning\l

In [3]:
winning_model = LogisticRegressionModel.load_model(trainable=False)

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
%%time
expected_return_df=compute_expected_return(compute_betting=wagering_stategies.race_betting_best_expected_return,
    source=SOURCE_PMU,
    code_pari="E_SIMPLE_GAGNANT",
    winning_model=winning_model)

C:\Users\Mathieu\Desktop\Projets\Benter\utils\import_data.py:221: DtypeWarning: Columns (35,46,47,61,62,63,64,87) have mixed types.Specify dtype option on import or set low_memory=False.
  rh_df = get_split_date(source=source, on_split=on_split)
  0%|                                                                                                                                                                                                        | 0/24690 [00:00<?, ?it/s]C:\Users\Mathieu\Anaconda3\envs\deeplearning\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
C:\Users\Mathieu\Desktop\Projets\Benter\utils\preprocess.py:124: SettingWithCopyWarning: 
A value is trying to be set on a

Wall time: 33min 32s


In [ ]:
%%time
best_proba_expected_return_df=compute_expected_return(compute_betting=wagering_stategies.race_betting_best_winning_proba,
    source=SOURCE_PMU,
    code_pari="E_SIMPLE_GAGNANT",
    winning_model=winning_model)

  0%|                                                                                                                                                                                                        | 0/24690 [00:00<?, ?it/s]C:\Users\Mathieu\Anaconda3\envs\deeplearning\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
C:\Users\Mathieu\Desktop\Projets\Benter\utils\preprocess.py:124: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ) / standard_

In [ ]:
%%time
pmp_expected_return_df = compute_expected_return(compute_betting=wagering_stategies.race_proportional_pari_mutual_proba,
    source=SOURCE_PMU,
    code_pari="E_SIMPLE_GAGNANT",
    winning_model=winning_model)

In [ ]:
expected_return_df.expected_return.mean()

In [ ]:
best_proba_expected_return_df.expected_return.mean()

In [ ]:
pmp_expected_return_df.expected_return.mean()

In [ ]:
for n_horse, n_horse_er_df in expected_return_df.groupby('n_horse'):
    print(n_horse,n_horse_er_df.expected_return.mean() , len(n_horse_er_df), f"{len(n_horse_er_df)/len(expected_return_df):.2%}" )


In [ ]:
for n_horse, n_horse_er_df in best_proba_expected_return_df.groupby('n_horse'):
    print(n_horse,n_horse_er_df.expected_return.mean() , len(n_horse_er_df), f"{len(n_horse_er_df)/len(best_proba_expected_return_df):.2%}" )


In [ ]:
for n_horse, n_horse_er_df in pmp_expected_return_df.groupby('n_horse'):
    print(n_horse,n_horse_er_df.expected_return.mean() , len(n_horse_er_df), f"{len(n_horse_er_df)/len(pmp_expected_return_df):.2%}" )


In [ ]:
expected_return_df.groupby('date').expected_return.agg(lambda s:(s>0).sum()) # TODO plot this, compute average day without winning distribution

In [ ]:
%%time
best_expected_return_scenario = compute_scenario(
    compute_betting_fun=wagering_stategies.race_betting_best_expected_return,
    source=SOURCE_PMU,
    code_pari="E_SIMPLE_GAGNANT",
    capital_fraction=0.01,
    winning_model=winning_model,
    verbose=True
) 

In [ ]:
%%time
best_expected_return_scenario_threshold_on_expected_return = compute_scenario(
    compute_betting_fun=wagering_factories.betting_on_best_expected_return_thresholded_expected_return_factory(expected_return_threshold=2.5),
    source=SOURCE_PMU,
    code_pari="E_SIMPLE_GAGNANT",
    capital_fraction=0.01,
    winning_model=winning_model,
) 

In [ ]:
%%time
best_expected_return_scenario_threshold_on_winning_proba = compute_scenario(
    compute_betting_fun=wagering_factories.betting_on_best_expected_return_thresholded_winning_probabilities_factory(minimum_winning_probabilities=0.05),
    source=SOURCE_PMU,
    code_pari="E_SIMPLE_GAGNANT",
    capital_fraction=0.01,
    winning_model=winning_model,
) 

In [ ]:
%%time
best_expected_return_scenario_threshold_on_expected_return_winning_proba = compute_scenario(
    compute_betting_fun=wagering_factories.betting_on_best_expected_return_thresholded_winning_probabilities_expected_returns_factory(minimum_winning_probabilities=0.05, expected_return_threshold=2.5),
    source=SOURCE_PMU,
    code_pari="E_SIMPLE_GAGNANT",
    capital_fraction=0.01,
    winning_model=winning_model,
) 

In [ ]:
plot_scenario(best_expected_return_scenario_threshold_on_expected_return)

In [ ]:
plot_scenario(best_expected_return_scenario_threshold_on_winning_proba)

In [ ]:
plot_scenario(best_expected_return_scenario_threshold_on_expected_return_winning_proba)

In [ ]:
import wagering_stategies

In [ ]:
%%time
not_max_odds_expected_return_df = compute_expected_return(compute_betting=wagering_stategies.race_betting_best_winning_proba_not_max_pari_mutual_proba,
    source=SOURCE_PMU,
    code_pari="E_SIMPLE_GAGNANT",
    winning_model=winning_model)



In [ ]:
not_max_odds_expected_return_df.expected_return.mean()

In [ ]:
for n_horse, n_horse_er_df in not_max_odds_expected_return_df.groupby('n_horse'):
    print(n_horse,n_horse_er_df.expected_return.mean() , len(n_horse_er_df), f"{len(n_horse_er_df)/len(not_max_odds_expected_return_df):.2%}" )